In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('seaborn')

In [ ]:
df = pd.read_csv('./dataset/train.csv')

### Explore

In [ ]:
df.columns

In [ ]:
for col in df.columns:
    print(df[col].value_counts().sort_values(ascending=False))

In [ ]:
df.isnull().sum().sort_values(ascending=False).head(20)

Dataset does not have werid Nan values, the attributes with high nan values are all intentional (eg. does not have pool so PoolQC is Nan)

#### Useful
Logically what might correlate lot with SalePrice: 

Category Type: MSSubClass, Neighborhood, OverallCond, 

Number: GrLivArea, YrSold, LotArea, YearBuilt, YearRemodAdd, PoolArea

In [ ]:
df.loc[:,["GrLivArea","SalePrice"]].sort_values('GrLivArea', ascending=False).head(30)

In [ ]:
df.loc[:,["GrLivArea","SalePrice"]].sort_values('GrLivArea', ascending=False).tail(30)

In [ ]:
df.groupby('Neighborhood')['SalePrice'].median()

In [ ]:
df.groupby('Neighborhood')['SalePrice'].describe()

In [ ]:
fig1 = plt.figure()

ax1_x = df.loc[:,'GrLivArea']
ax1_y = df.loc[:,'SalePrice']

ax1 = fig1.add_subplot(211)
ax1.scatter(ax1_x, ax1_y , color='lightblue', linewidth=3, label="Above Ground Living Area v Price")
ax1.set( xlabel="Living Area", ylabel="Sale Price")
ax1.legend()

#???
#why does df.groupby('Neighborhood')['SalePrice'].median().plot.bar() work

ax2 = fig1.add_subplot(212)
ax2_y = df.groupby('Neighborhood')['SalePrice'].median().values
ax2_x = df.groupby('Neighborhood')['SalePrice'].median().index
ax2.bar(ax2_x, ax2_y)
ax2.tick_params(axis='x', rotation=80)

plt.tight_layout()



#### Observation from graphs

Neighborhood is not a great predictor but there is a trend with above ground living area

In [ ]:
df.sort_index()
dfT = df[['GrLivArea', 'SalePrice']]
dfT

In [ ]:
X_train = dfT.loc[:,'GrLivArea'].values.reshape(-1,1)
Y_train = dfT.loc[:,'SalePrice'].values.reshape(-1,1)

#this fails! 
# Y_train = dfT.loc[:,'SalePrice'].values
# Y_train.shape gives (1460,)




#### To Do:

Try Create new Feature: Living Area: TotalBsmtSF - BsmtUnfSF + GrLivArea

See if predict better

Also, do test train split since don't have answer to the test.csv data from Kaggle

### Regression Fitting

In [ ]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

In [ ]:
print(regressor.intercept_)


In [ ]:
testing_df = pd.read_csv('./dataset/test.csv')
testing_df = testing_df[['Id','GrLivArea']]
testing_df


In [ ]:
X_test = testing_df.loc[:,'GrLivArea'].values.reshape(-1,1)
y_pred = regressor.predict(X_test)
y_pred

In [ ]:
# fit needs and returns a 2D array [[1,2,3]] so need to convert in and out

y_pred_flat = y_pred.ravel()
my_ans = pd.DataFrame({'Id': testing_df['Id'].values, 'SalePrice': y_pred_flat})
my_ans

In [ ]:
my_ans.to_csv('my_ans.csv',index=False)